In [2]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
FB_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/FB_predictions.csv", index_col=0)


In [4]:
FB_predictions.head()

,Date,Predictions
0,2019-07-31,0
1,2019-06-28,0
2,2019-05-31,0
3,2019-04-30,1
4,2019-03-29,0


In [5]:
FB_predictions.Date = pd.to_datetime(FB_predictions.Date, format="%Y-%m-%d")

In [6]:
FB_predictions['Predictions'] = FB_predictions['Predictions'].shift(-1)


In [7]:
FB_predictions = FB_predictions.dropna()


In [8]:
FB_predictions = FB_predictions[::-1]

In [9]:
FB_predictions.head(10)

,Date,Predictions
83,2012-08-31,1.0
82,2012-09-28,1.0
81,2012-10-31,1.0
80,2012-11-30,0.0
79,2012-12-31,1.0
78,2013-01-31,0.0
77,2013-02-28,1.0
76,2013-03-28,1.0
75,2013-04-30,1.0
74,2013-05-31,0.0


In [10]:
FB_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/FB/FB_monthly.csv", index_col =0)

In [11]:
FB_monthly.columns = ['Open','High','Low','Close','Volume']

In [12]:
FB_predictions = FB_predictions.set_index("Date")

In [13]:
FB_monthly.head()

,Open,High,Low,Close,Volume
2019-08-15,194.17,198.4700,179.3100,182.59,172260200
2019-07-31,195.21,208.6749,191.9300,194.23,351062500
2019-06-28,175.00,194.5300,160.8316,193.00,426428600
2019-05-31,194.78,196.1770,177.1600,177.47,286821500
2019-04-30,167.83,198.4800,167.2790,193.40,381985300


In [14]:
FB_final_df = pd.merge(FB_predictions, FB_monthly, right_index=True, left_index=True)

In [15]:
FB_final_df.index = pd.to_datetime(FB_final_df.index, format ="%Y-%m-%d")

In [17]:
equity_dict = dict()

In [18]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] and (position.is_short!=True) == 0.0 :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [19]:
bt = Backtest(FB_final_df, Binary_Classification_1d, cash=10000, commission=.01)
bt.run()



Start                          2012-08-31 00:00:00
End                            2019-07-31 00:00:00
Duration                        2525 days 00:00:00
Exposure [%]                               97.5842
Equity Final [$]                            259135
Equity Peak [$]                             259135
Return [%]                                 2491.35
Buy & Hold Return [%]                       975.59
Max. Drawdown [%]                         -19.5158
Avg. Drawdown [%]                         -5.53498
Max. Drawdown Duration           397 days 00:00:00
Avg. Drawdown Duration           101 days 00:00:00
# Trades                                        81
Win Rate [%]                               64.1975
Best Trade [%]                             33.3725
Worst Trade [%]                           -14.9528
Avg. Trade [%]                             3.62107
Max. Trade Duration               33 days 00:00:00
Avg. Trade Duration               31 days 00:00:00
Expectancy [%]                 

In [20]:
bt.plot(superimpose=False, plot_volume=False)

In [21]:
df = pd.DataFrame.from_dict([equity_dict])

In [22]:
df = df.T

In [23]:
df.columns = ['Equity']

In [24]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/FB_CNN_Equity.csv")

In [25]:
df

,Equity
2012-09-28,10000.000000
2012-10-31,9541.605195
2012-11-30,12630.586552
2012-12-31,13226.640750
2013-01-31,14486.772584
2013-02-28,16226.063826
2013-03-28,15401.735852
2013-04-30,16686.361557
2013-05-31,14601.483453
2013-06-28,14154.584033
